# Image Contrast Enhancement

Contents:
- Full-Scale Contrast Stretch
- Gamma Transforms
- Histogram Equalisation
- Adaptive Histogram Equalisation (CLAHE)

Author: Abijith J. Kamath, IISc.
https://kamath-abhijith.github.io

In [ ]:
import numpy as np

from skimage import io
from matplotlib import pyplot as plt
from matplotlib import rcParams
from matplotlib import style

## Full-Scale Constrast Stretch

Let $I$ be an $8$-bit image. The transformation:
$$
    I := \frac{255}{b-a}(I - a),
$$
where $b = \max(I)$ and $a = \min(I)$, transforms the image such that $\min(I) = 0$ and $\max(I) = 255$, thus, in some sense improving the contrast of the image.

In [ ]:
## FUNCTION DEFINITION: FULL-SCALE CONTRAST STRETCH
def fscs(image):
    image = np.double(image)

    return (255/(np.max(image)-np.min(image)))*(image - np.min(image))

## Gamma Transform

Let $I$ be an $8$-bit, normalised image. The gamma transformation:

$$
I := I^{\gamma}, \; \gamma > 0.
$$
The contrast of the trasnformed image is better when $0<\gamma<1$.

In [ ]:
## FUNCTION DEFINITION: GAMMA TRANSFORM

def gamma_transform(image, gamma=0.5):
    return ((np.double(image)/255.0)**gamma)*255.0

## Histogram Equalisation
Let $X$ be a continuous random variable with an invertible distribution function $F$. Define $Y = F(X)$. Since, $F(x) = [0,1]$, $0\leq Y \leq 1$, with the distribution function:
$$
F_Y(y) = Pr[Y\leq y] = Pr[F(X)\leq y] = Pr[X\leq F^{-1}(y)] = F(F^{-1}(y)) = y,
$$
i.e., Y has a uniform distribution. Let $I$ be an $8$-bit image with the normalised histogram $p$. The histogram equalisation:
$$
J(i,j) = \sum_{k=0}^{I(i,j)} p(k),
$$
is a discrete approximation to the continous transform, i.e., the transformed image $J$ has a better (more uniform) contrast.

In [ ]:
## FUNCTION DEFINITION: HISTOGRAM EQUALISATION

def hist_eq(image):
    [m,n] = image.shape

    [image_hist, _] = np.histogram(image, bins=np.arange(0,255,1))
    image_density = image_hist/(m*n)

    sum_image_density = np.cumsum(image_density)
    sum_image_density = np.append(sum_image_density, [0,0])
    hist_eq_image = np.zeros((m,n), dtype=np.int)
    for i in range(m):
        for j in range(n):
            hist_eq_image[i,j] = 255.0*sum_image_density[image[i,j]]

    return hist_eq_image

## Saturated Contrast Stretching
Linear Constrast Stretch is applied to the image after setting some pixels to zero and some pixels to one. Since errors in discrete implementation gives a histogram that is not uniform, this method sets the histogram to full scale and then performs linear stretching.


In [ ]:
## FUNCTION DEFINITION: SATURATED CONTRAST STRETCHING

def saturated_contrast_stretching(image, thresholds=(10, 90)):
    
    llim = thresholds[0]*255.0/100
    ulim = thresholds[1]*255.0/100

    zero_image = (image<=llim)*1.0
    ones_image = (image<=ulim)*1.0

    set_zero_image = image*np.logical_not(zero_image)
    set_ones_image = set_zero_image*np.logical_not(ones_image) + np.logical_and(set_zero_image, ones_image)*255.0
    
    return fscs(set_ones_image)

In [ ]:
## READ/DISPLAY IMAGES WITH THEIR HISTOGRAMS

image1 = io.imread("LowLight_1.png", 0)
image2 = io.imread("LowLight_2.png", 0)
image3 = io.imread("LowLight_3.png", 0)
image4 = io.imread("Hazy.png", 0)
image5 = io.imread("StoneFace.png", 0)

numbins = 255
[image1_hist, bins] = np.histogram(image1, bins=np.arange(0,numbins,1))
[image2_hist, bins] = np.histogram(image2, bins=np.arange(0,numbins,1))
[image3_hist, bins] = np.histogram(image3, bins=np.arange(0,numbins,1))
[image4_hist, bins] = np.histogram(image4, bins=np.arange(0,numbins,1))
[image5_hist, bins] = np.histogram(image5, bins=np.arange(0,numbins,1))

In [ ]:
## PLOTS
style.use('classic')

rcParams['text.usetex'] = True
rcParams.update({'font.size': 10})
rcParams['text.latex.preamble'] = [r'\usepackage{tgheros}'] 

In [ ]:
## FULL-SCALE CONTRAST STRETCH

image1_fscs = fscs(image1)
image2_fscs = fscs(image2)

[image1_fscs_hist, bins] = np.histogram(image1_fscs, bins=np.arange(0,numbins,1))
[image2_fscs_hist, bins] = np.histogram(image2_fscs, bins=np.arange(0,numbins,1))

fig, plts = plt.subplots(1,3,figsize=(15,6))
plts[0].imshow(image1, cmap='gray')
plts[0].set_ylabel(r"Low Light 1")
plts[0].set_xlabel(r"Original Image")

plts[1].imshow(image1_fscs, cmap='gray')
plts[1].set_xlabel(r"Linear Constrast Stretching")

plts[2].plot(image1_fscs_hist, '-', color='green', label=r'Linear Contrast Stretching')
plts[2].plot(image1_hist, '-', color='blue', label=r'Original')
plts[2].set_xlim([0, 255])
plts[2].legend()
plts[2].grid('on')
plt.savefig('/Users/abhijith/Desktop/TECHNOLOGIE/Courses/E9 241 Digital Signal Processing/Assignments/Assignment_2/Answers/figures/lowlight1_fscs.eps', format='eps')


ffig, plts = plt.subplots(1,3,figsize=(15,6))
plts[0].imshow(image2, cmap='gray')
plts[0].set_ylabel(r"Low Light 2")
plts[0].set_xlabel(r"Original Image")

plts[1].imshow(image2_fscs, cmap='gray')
plts[1].set_xlabel(r"Linear Constrast Stretching")

plts[2].plot(image2_fscs_hist, '-', color='green', label=r'Linear Contrast Stretching')
plts[2].plot(image2_hist, '-', color='blue', label=r'Original')
plts[2].set_xlim([0, 255])
plts[2].legend()
plts[2].grid('on')
plt.savefig('/Users/abhijith/Desktop/TECHNOLOGIE/Courses/E9 241 Digital Signal Processing/Assignments/Assignment_2/Answers/figures/lowlight2_fscs.eps', format='eps')

plt.show()

In [ ]:
## GAMMA TRANSFORMATION

image1_gamma = gamma_transform(image1)
image2_gamma = gamma_transform(image2)
image4_gamma = gamma_transform(image4)

[image1_gamma_hist, bins] = np.histogram(image1_gamma, bins=np.arange(0,numbins,1))
[image2_gamma_hist, bins] = np.histogram(image2_gamma, bins=np.arange(0,numbins,1))
[image4_gamma_hist, bins] = np.histogram(image4_gamma, bins=np.arange(0,numbins,1))

fig, plts = plt.subplots(1,3,figsize=(15,6))
plts[0].imshow(image1, cmap='gray')
plts[0].set_ylabel(r"Low Light 1")
plts[0].set_xlabel(r"Original Image")

plts[1].imshow(image1_gamma, cmap='gray')
plts[1].set_xlabel(r"Gamma Transform")

plts[2].plot(image1_gamma_hist, '-', color='green', label=r'Gamma Transform')
plts[2].plot(image1_hist, '-', color='blue', label=r'Original')
plts[2].set_xlim([0, 255])
plts[2].legend()
plts[2].grid('on')
plt.savefig('/Users/abhijith/Desktop/TECHNOLOGIE/Courses/E9 241 Digital Signal Processing/Assignments/Assignment_2/Answers/figures/lowlight1_gamma.eps', format='eps')

fig, plts = plt.subplots(1,3,figsize=(15,6))
plts[0].imshow(image2, cmap='gray')
plts[0].set_ylabel(r"Low Light 2")
plts[0].set_xlabel(r"Original Image")

plts[1].imshow(image2_gamma, cmap='gray')
plts[1].set_xlabel(r"Gamma Transform")

plts[2].plot(image2_gamma_hist, '-', color='green', label=r'Gamma Transform')
plts[2].plot(image2_hist, '-', color='blue', label=r'Original')
plts[2].set_xlim([0, 255])
plts[2].legend()
plts[2].grid('on')
plt.savefig('/Users/abhijith/Desktop/TECHNOLOGIE/Courses/E9 241 Digital Signal Processing/Assignments/Assignment_2/Answers/figures/lowlight2_gamma.eps', format='eps')

fig, plts = plt.subplots(1,3,figsize=(15,6))
plts[0].imshow(image4, cmap='gray')
plts[0].set_ylabel(r"Hazy")
plts[0].set_xlabel(r"Original Image")

plts[1].imshow(image4_gamma, cmap='gray')
plts[1].set_xlabel(r"Gamma Transform")

plts[2].plot(image4_gamma_hist, '-', color='green', label=r'Gamma Transform')
plts[2].plot(image4_hist, '-', color='blue', label=r'Original')
plts[2].set_xlim([0, 255])
plts[2].legend()
plts[2].grid('on')
plt.savefig('/Users/abhijith/Desktop/TECHNOLOGIE/Courses/E9 241 Digital Signal Processing/Assignments/Assignment_2/Answers/figures/hazy_gamma.eps', format='eps')

plt.show()

In [ ]:
## HISTOGRAM EQUALISATION

image2_histeq = hist_eq(image2)
image3_histeq = hist_eq(image3)
image4_histeq = hist_eq(image4)
image5_histeq = hist_eq(image5)

[image2_histeq_hist, bins] = np.histogram(image2_histeq, bins=np.arange(0,numbins,1))
[image3_histeq_hist, bins] = np.histogram(image3_histeq, bins=np.arange(0,numbins,1))
[image4_histeq_hist, bins] = np.histogram(image4_histeq, bins=np.arange(0,numbins,1))
[image5_histeq_hist, bins] = np.histogram(image5_histeq, bins=np.arange(0,numbins,1))

fig, plts = plt.subplots(1,3,figsize=(15,6))
plts[0].imshow(image2, cmap='gray')
plts[0].set_ylabel(r"Low Light 2")
plts[0].set_xlabel(r"Original Image")

plts[1].imshow(image2_histeq, cmap='gray')
plts[1].set_xlabel(r"Histogram Equalisation")

plts[2].plot(image2_histeq_hist, '-', color='green', label=r'Histogram Equalisation')
plts[2].plot(image2_hist, '-', color='blue', label=r'Original')
plts[2].set_xlim([0, 255])
plts[2].legend()
plts[2].grid('on')
plt.savefig('/Users/abhijith/Desktop/TECHNOLOGIE/Courses/E9 241 Digital Signal Processing/Assignments/Assignment_2/Answers/figures/lowlight2_histeq.eps', format='eps')

fig, plts = plt.subplots(1,3,figsize=(15,6))
plts[0].imshow(image3, cmap='gray')
plts[0].set_ylabel(r"Low Light 3")
plts[0].set_xlabel(r"Original Image")

plts[1].imshow(image3_histeq, cmap='gray')
plts[1].set_xlabel(r"Histogram Equalisation")

plts[2].plot(image3_histeq_hist, '-', color='green', label=r'Histogram Equalisation')
plts[2].plot(image3_hist, '-', color='blue', label=r'Original')
plts[2].set_xlim([0, 255])
plts[2].legend()
plts[2].grid('on')
plt.savefig('/Users/abhijith/Desktop/TECHNOLOGIE/Courses/E9 241 Digital Signal Processing/Assignments/Assignment_2/Answers/figures/lowlight3_histeq.eps', format='eps')

fig, plts = plt.subplots(1,3,figsize=(15,6))
plts[0].imshow(image4, cmap='gray')
plts[0].set_ylabel(r"Hazy")
plts[0].set_xlabel(r"Original Image")

plts[1].imshow(image4_histeq, cmap='gray')
plts[1].set_xlabel(r"Histogram Equalisation")

plts[2].plot(image4_histeq_hist, '-', color='green', label=r'Histogram Equalisation')
plts[2].plot(image4_hist, '-', color='blue', label=r'Original')
plts[2].set_xlim([0, 255])
plts[2].legend()
plts[2].grid('on')
plt.savefig('/Users/abhijith/Desktop/TECHNOLOGIE/Courses/E9 241 Digital Signal Processing/Assignments/Assignment_2/Answers/figures/hazy_histeq.eps', format='eps')

fig, plts = plt.subplots(1,3,figsize=(15,6))
plts[0].imshow(image5, cmap='gray')
plts[0].set_ylabel(r"Stone Face")
plts[0].set_xlabel(r"Original Image")

plts[1].imshow(image5_histeq, cmap='gray')
plts[1].set_xlabel(r"Histogram Equalisation")

plts[2].plot(image5_histeq_hist, '-', color='green', label=r'Histogram Equalisation')
plts[2].plot(image5_hist, '-', color='blue', label=r'Original')
plts[2].set_xlim([0, 255])
plts[2].legend()
plts[2].grid('on')
plt.savefig('/Users/abhijith/Desktop/TECHNOLOGIE/Courses/E9 241 Digital Signal Processing/Assignments/Assignment_2/Answers/figures/stoneface_histeq.eps', format='eps')

plt.show()

In [ ]:
## SATURATED CONTRAST STRETCHING

image1_scs = saturated_contrast_stretching(image1)

[image1_scs_hist, bins] = np.histogram(image1_scs, bins=np.arange(0,numbins,1))

fig, plts = plt.subplots(1,2,figsize=(20, 20))
plts[0].imshow(image1_scs, cmap='gray')
plts[1].plot(image1_scs_hist)

plt.show()